In [ ]:
# Import pandas for data manipulation (tables, reading CSV files)
import pandas as pd  

# Import numpy for numerical operations (arrays, linspace, etc.)
import numpy as np  

# Import matplotlib for plotting graphs (histograms, boxplots, etc.)
import matplotlib.pyplot as plt  

# Import statistical tools from scipy (tests, distribution fitting, Q-Q plots)
from scipy import stats  

# Optional: show all columns when printing a dataframe
pd.set_option("display.max_columns", 100)

In [ ]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("students_ai_usage.csv")

# Print confirmation message
print("Dataset successfully loaded!")

# Print number of rows and columns
print("Shape of dataset:", df.shape)

# Print column names
print("Columns in dataset:")
print(df.columns.tolist())

# Show first 5 rows of the dataset
df.head()

In [ ]:
# These columns should be numeric for statistical analysis
numeric_columns = [
    "age",
    "study_hours_per_day",
    "grades_before_ai",
    "grades_after_ai",
    "daily_screen_time_hours"
]

# Convert each column to numeric
# If conversion fails, value becomes NaN
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Display how many missing values exist
print("Missing values per numeric column:")
df[numeric_columns].isna().sum()

In [ ]:
# We select grades_after_ai as the main variable for distribution analysis
target = "grades_after_ai"

# Remove missing values from this column
x = df[target].dropna()

# Print how many valid observations we have
print("Target variable:", target)
print("Number of valid observations:", len(x))

In [ ]:
# Calculate mean (average)
mean = x.mean()

# Calculate median (middle value)
median = x.median()

# Calculate standard deviation (spread of data)
std = x.std(ddof=1)

# Calculate variance (std squared)
variance = x.var(ddof=1)

# Calculate skewness (asymmetry)
skewness = stats.skew(x, bias=False)

# Calculate kurtosis (tail heaviness)
kurtosis = stats.kurtosis(x, fisher=True, bias=False)

# Put results into a table
summary_table = pd.DataFrame({
    "Metric": ["Count", "Mean", "Median", "Std Dev", "Variance", "Skewness", "Kurtosis"],
    "Value": [len(x), mean, median, std, variance, skewness, kurtosis]
})

summary_table

In [ ]:
# Create new figure
plt.figure()

# Plot histogram with 20 bins
plt.hist(x, bins=20)

# Add labels and title
plt.title("Histogram of grades_after_ai")
plt.xlabel("grades_after_ai")
plt.ylabel("Frequency")

# Show plot
plt.show()

In [ ]:
# Create new figure
plt.figure()

# Create horizontal boxplot
plt.boxplot(x, vert=False)

# Add labels
plt.title("Boxplot of grades_after_ai")
plt.xlabel("grades_after_ai")

# Show plot
plt.show()

In [ ]:
# Create new figure
plt.figure()

# Compare sample quantiles to theoretical normal quantiles
stats.probplot(x, dist="norm", plot=plt)

# Add title
plt.title("Q-Q Plot (Normal Distribution Check)")

# Show plot
plt.show()

In [ ]:
# Sample up to 5000 observations for Shapiro test
x_sample = x.sample(min(len(x), 5000), random_state=42)

# Perform Shapiro-Wilk test (H0: data is normal)
shapiro_stat, shapiro_p = stats.shapiro(x_sample)

# Perform D’Agostino K² test (H0: data is normal)
dagostino_stat, dagostino_p = stats.normaltest(x)

print("Shapiro-Wilk Test p-value:", shapiro_p)
print("D’Agostino Test p-value:", dagostino_p)

# Interpretation
if dagostino_p > 0.05:
    print("Data appears approximately normally distributed.")
else:
    print("Data is NOT normally distributed.")

In [ ]:
# Convert to numpy array
data = x.values

# Estimate normal distribution parameters (mu and sigma)
mu, sigma = stats.norm.fit(data)

# Compute log-likelihood
log_likelihood_norm = np.sum(stats.norm.logpdf(data, mu, sigma))

# Number of parameters (mu and sigma)
k_norm = 2

# Calculate AIC for normal distribution
aic_norm = 2 * k_norm - 2 * log_likelihood_norm

print("Normal Distribution Fit")
print("Mean (mu):", mu)
print("Std Dev (sigma):", sigma)
print("AIC:", aic_norm)

In [ ]:
# Lognormal only works for positive values
positive_data = data[data > 0]

# Estimate lognormal parameters
shape, loc, scale = stats.lognorm.fit(positive_data, floc=0)

# Compute log-likelihood
log_likelihood_lognorm = np.sum(stats.lognorm.logpdf(positive_data, shape, loc=0, scale=scale))

# Number of parameters
k_lognorm = 2

# Calculate AIC
aic_lognorm = 2 * k_lognorm - 2 * log_likelihood_lognorm

print("Lognormal Distribution Fit")
print("Shape:", shape)
print("Scale:", scale)
print("AIC:", aic_lognorm)